# Constellation Generation

## Quick guide:
* Initialize the constellation by executing the first cell and providing metadata in the second cell (1)
* Create a shell by providing the 4 shell arguments, and further parameters (extra params) if necessary (2.1).
Store the temporary shell data by executing the cell (2.2)
* Turn satellites into position and velocity vectors by executing cell (3)
* Write the files by executing cell (4) and save them by executing cell (5)

The following guide contains more detail

## Detailed guide:

### 0. Execute the first cell to import libraries and initialize variables

### 1. Initialize Constellation and reset planet_list and other lists
Provide the constellation name, the time of launch `startTime` as well as
the time it takes to fully deploy the constellation `duration` and execute the cell. 
`startTime` can be specified as a date string with the format "YYYY-MM-DD". `duration` is specified as a string that contains a number of days followed by the 'd' character. If the variables are assigned plain integers, the values will be interpreted as simulation iterations.
### 2.1 Generate shell = ( altitude, inclination, nPlanes, nSats )
The execution of this cell creates one temporary shell (each satellite is described by orbital elements) that is not yet a part of the constellation. It is possible to visualize the generated satellites by executing the cell `Plot current shell` before adding it to the constellation. Append the shell to the constellation by executing cell `2.2`. Each time the code of `2.1` is run, the temporary data is overridden by the new data.

Create a shell by specifying the parameters of an orbital shell which are the 
following:
* `altitude` the altitude of each satellite in kilometers, relative to the
earths surface
* `inclination` the inclination of each orbital plane in degrees
* `nPlanes` the number of orbital planes
* `nSats` the number of satellites in each plane

In some cases it is necessary to specify more parameters: `offsetM`, `argPeriapsis`,
`startingW`, and `W_area`. It is best to modify them only if necessary and to use
the default values otherwise. Refer to the end of the notebook to see their usage
information (additional parameters).

### 2.2 Append to elements_list and other
Append the temporary set of satellites (elements_list_tmp) to the
constellation (elements_list). The same happens to lists with the shells
meta information. At this stage elements_list entries are the orbital elements of the satellite.

### 3. Create position and velocity vectors
Execute the code in order to create position and velocity vectors from the list of orbital elements. They are stored in the 'satellites' variable.

### 4. Plot and store results
Prepare .csv and .yaml output and simultaneously plot the constellation by
executing this cell.

### 5. Save Constellation as files in directory
Create a directory with the same name that was specified in `1` in the variable
`constellation_name` in the `data` directory of the project. It contains three
files: Two .csv files with positions and velocities and a .yaml file with the
metadata of the constellation.

## 0. Import and initialize variables

In [ ]:
### Imports
%load_ext autoreload
%autoreload 2

# Append main folder
import sys
sys.path.append("../")

import pykep as pk
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import yaml

starting_t = pk.epoch_from_string('2022-01-01 00:00:00.000')

# 1. Initialize Constellation and reset planet_list and other lists

In [ ]:
#---Constellation data----------------------------------------------------------------------------------------------
constellation_name = "Constellation"
startTime = 0
duration = 0
#-------------------------------------------------------------------------------------------------------------------
nShells = 0
elements_list = []
shell_list = []
xparams_list = []
print("Planet list is now empty")

# 2.1 Generate shell = ( altitude, inclination, nPlanes, nSats )

In [ ]:
# creates a temporary shell that has to be appended to planet_list in cell "Append to planet_list and other"
# to be stored and not get overwritten (this also concerns additional shell data shell_list, xparams_list)

#---input-----------------------------------------------------------------------------------------------------------
altitude = 700
inclination = 40.0
nPlanes = 20
nSats = 10
#-------------------------------------------------------------------------------------------------------------------
#---extra-params----------------------------------------------------------------------------------------------------

#offsetM = offset for Mean Anomaly added after each plane (relative phasing)
# walker constellation: offsetM = F * 360 / (nPlanes * nSats) ; F element {0, ... , nPlanes - 1}
offsetM = 0    # default 0

#argPeriapsis = argument of periapsis
# starting point of satellite placement for each plane
# argPeriapsis = pi avoids collisions in planes that intersect at reference plane
argPeriapsis = math.pi  # default math.pi 

#startingW = offset for W that is not accumulating (W = longitude of ascending node)
# formula for overlapping shells (same altitude, same inclination):
# (360 / G) / 2 ; G = smallest common multiple of the overlapping nPlanes
startingW = 0  # default 0

# W_area: orbital planes are distributed evenly within range [startingW,startingW + maximumW)
W_area = 360     # default 360
#-------------------------------------------------------------------------------------------------------------------

minW = startingW
maxW = W_area + startingW

a = altitude * 1000 + 6371000    # in [m], earth radius included
e = 0
i = inclination * pk.DEG2RAD
W = pk.DEG2RAD * minW
w = argPeriapsis * pk.DEG2RAD       
M = 0             

plane_count = 0

when = starting_t
mu_central_body = pk.MU_EARTH
mu_self = 1
radius = 1
safe_radius = 1

pStep = pk.DEG2RAD * W_area / nPlanes  # W goes from startingW to startingW+W_area
sStep = 2 * math.pi / nSats              # M goes from 0° to 360°
sExtraStep = pk.DEG2RAD*offsetM

planet_list_tmp = []
elements_list_tmp = []
for x in range(nPlanes):
    for y in range(nSats):
        planet_list_tmp.append(pk.planet.keplerian(when,[a,e,i,W,w,M], mu_central_body, mu_self,radius, safe_radius ,"sat"))
        elements_list_tmp.append([a,e,i,W,w,M])
        M = M + sStep
    plane_count = plane_count + 1
    W = W + pStep
    M = plane_count * sExtraStep     #equals 0 + count*0 = 0 in the usual case
    
shell = (altitude,inclination,nPlanes,nSats)
xparams = (offsetM,argPeriapsis,startingW,W_area)

print("Added " + str(len(elements_list_tmp)) + " planets")

# 2.2 Append to planet_list and other

In [ ]:
# stores and appends temporary shell in planet_list (including lists with additional data)
nShells += 1
elements_list = elements_list + elements_list_tmp
shell_list.append(shell)
xparams_list.append(xparams)
print("Added " + str(len(elements_list_tmp)) + " to planet_list now totalling " + str(len(elements_list)) + " planets.")

### Plot current shell

In [ ]:
fig = plt.figure(figsize=(6,6),dpi=100)
ax = plt.axes(projection='3d')
for i in range (nPlanes*nSats):
    pk.orbit_plots.plot_planet(planet_list_tmp[i],axes=ax,s=20)

# 3. Create position and velocity vectors

In [ ]:
satellites = []
for elements in elements_list:
    pos,v = pk.par2ic(elements,pk.MU_EARTH)
        
    # convert to km and numpy
    pos = np.asarray(pos) / 1000.0 
    v = np.asarray(v) / 1000.0
        
    satellites.append((pos,v))
        
print("Successfully propagated ",len(satellites)," satellites.")

# 4. Plot and store results

In [ ]:
fig = plt.figure(figsize=(6,6),dpi=100)
ax = plt.axes(projection='3d');
# pos , v (.csv)
positions = np.array([pos for pos,_ in satellites])
velocities = np.array([v for _,v in satellites])
ax.scatter(positions[:,0],positions[:,1],positions[:,2],".",alpha=0.25)
# parameters (.yaml)
constellation_cfg = {
    "constellation": {
        "name": constellation_name,
        "startTime": startTime,
        "duration": duration,
        "nShells": nShells
    }
}
for i in range(nShells):
    constellation_cfg["shell" + str(i+1)] = {
        "altitude": shell_list[i][0],
        "inclination": shell_list[i][1],
        "nPlanes": shell_list[i][2],
        "nSats": shell_list[i][3]
    }
for i in range(nShells):
    constellation_cfg["special" + str(i+1)] = {
        "offsetM": xparams_list[i][0],
        "argPeriapsis": xparams_list[i][1],
        "startingW": xparams_list[i][2],
        "W_area": xparams_list[i][3]
    }

# 5. Save Constellation as files in directory

In [ ]:
# creates directory named as the constellation_name value within the data folder and stores output there

os.mkdir("../../data/" + constellation_name);

np.savetxt("../../data/" + constellation_name + "/pos_"+constellation_name+".csv",positions,delimiter=",")
np.savetxt("../../data/" + constellation_name + "/v_"+constellation_name+".csv",velocities,delimiter=",")

with open("../../data/" + constellation_name + "/shells_"+constellation_name+".yaml", "w") as fh:  
    yaml.dump(constellation_cfg, fh,sort_keys=False)
    fh.close()

### Additional parameters
### offsetM
`offsetM` is the step size for an offset that is added to each satellite's mean anomaly. The offset accumulates, meaning it increases by `offsetM` each time a new plane is generated. In a Walker Constellation, 
which is a typical model for constellations, the parameter is assigned a value of the following form:

`offsetM = F * 360 / (nPlanes*nSats)` 

Note: F is a natural number: F is element of {0, ... , nPlanes - 1}

Note: the property of a set phasing difference
between neighboring planes gets lost if the shell is added over time (`duration` > 0)



### argPeriapsis
`argPeriapsis` corresponds to the orbital element "argument of periapsis". Choosing an irrational number
prevents satellites from two symmetrical planes to have the same position.

### startingW
`startingW` is the base value for the shell's longitude of ascending node (W). The parameter can be utilized if planes of different shells overlap. This occurs if shells have the same altitude and the same or similar inclination.

The shells should follow the formula:

`(360/G)*(i/N)`

Note: G stands for the smallest common denominator of the nPlanes parameters of
each shell

Note: N is the number of shells overlapping

Note: i is an index in {0,...,N-1}. each shell should have a distinct i

### W_area
this parameter determines which degree the planes of the shell span. The longitudes of ascending node are distributed in the range \[startingW , startingW + W_area\].